In [ ]:
!pwd

In [ ]:
import torch
import torch.nn as nn  #import neural network class
import torch.nn.functional as F  #loss functions
from torch.utils.data import DataLoader   #for batch seperation
from torchvision.transforms import ToTensor  ####convert tensors to vectors
from torchvision import datasets
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from torchvision import datasets, models, transforms


**> **transform data to tensor and normalize the data also do image resize ****

In [ ]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.50,.50),(0.50,.50,.5)),transforms.Resize((180,180)),])#  mean and standard deviation to be passed in normalization

In [ ]:
import cv2
from matplotlib import pyplot as plt

In [ ]:
train_img = datasets.ImageFolder('/kaggle/input/alien-vs-predator-images/data/train',transform = transform)
valid_img  = datasets.ImageFolder('/kaggle/input/alien-vs-predator-images/data/validation',transform = transform)

In [ ]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()
# %load_ext tensorboard

> **display few random data points**

In [ ]:
figure = plt.figure(figsize=(10, 10))
cols, rows = 3, 3
for i in range(1, cols * rows + 1):
    sample_idx = torch.randint(len(train_img), size=(1,)).item()
    img, label = train_img[sample_idx]
    figure.add_subplot(rows, cols, i)
    img = img.permute(1, 2, 0).numpy()
    #plt.title(lab[label])
    plt.axis("off")
    plt.imshow(img)
plt.show()

In [ ]:
!pwd

In [ ]:
%load_ext tensorboard
%tensorboard --logdir = "/kaggle/working" --port = 6006

In [ ]:
#  tensorboard dev
#     upload --logdir logs \
#     --name "My latest experiment" \
#     --description "(optional) Simple comparison of several hyperparameters"

In [ ]:
# !tensorboard dev upload \
#   --logdir "/kaggle/working" \
#   --name "My latest expariment" \
#   --description "simple comparison of several hyperparameters" \
#   --one_shot
# response = input()

> ****do batches****

In [ ]:
batch_size = 64
traindata_loader = torch.utils.data.DataLoader(train_img,batch_size=batch_size,shuffle = True)
testdata_loader = torch.utils.data.DataLoader(valid_img,batch_size=batch_size,shuffle = True)

In [ ]:
len(traindata_loader),len(testdata_loader)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

In [ ]:
for batch,(X,y) in enumerate(traindata_loader):
    print(len(X))
    break

# modify Resnet50 Architecture

In [ ]:
model = models.resnet50(pretrained=True)
# limiting the weights to be constant during training
for param in model.parameters():
    param.requires_grad = False
model.fc = nn.Sequential(nn.Linear(2048, 256),nn.Dropout2d(.3),nn.ReLU(inplace=True),nn.Linear(256,64),nn.Dropout2d(.3),nn.Linear(64,2))

In [ ]:
model = model.to(device)

In [ ]:
model

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr= 0.1,weight_decay = 0.01)

> **adding graph to tensorboard**

In [ ]:
sample_input = torch.randn(batch_size, 3, 32, 32).to('cuda' if torch.cuda.is_available() else 'cpu')
writer.add_graph(model,sample_input)
writer.close()


In [ ]:
for X,y in traindata_loader:
    print(X.shape)
  #X = X.to(device)
    #print(model(X))
    break

In [ ]:
for (X,y) in traindata_loader:
    print(X[0].shape)
    X = X.to(device)
    print(model(X))
    break

In [ ]:
import time
start = time.time()

# Train

In [ ]:
def train(data,model,optim,loss_,epoch):
    total_loss = 0.0
    model.train()
    for batch,(X,y) in enumerate(data):
        #print(batch)
        X = X.to(device)
        y = y.to(device)


        pred = model(X)

        loss = loss_(pred,y)
        loss.backward()
        optim.step()
        optim.zero_grad()
        total_loss += loss.item()
#         if batch % 10 == 0:
#             #print(batch)
#             print(f'batch level loss of {batch} is  :{loss}')
#             #bat_loss = total_loss / (batch+1)*64
#             #print(f'avg of all losses:{bat_loss}')
    avg_loss = total_loss / len(data)
    writer.add_scalar("Training Loss", avg_loss, epoch)
    print(f'training loss of {batch} is {avg_loss}')

# Test 

In [ ]:
def test(model,data,loss_,epoch):
    
    model.eval()
    test_error,correct = 0,0
    size  = len(data.dataset)

    with torch.no_grad():

        for batch,(X,y) in enumerate(data):
            
            X = X.to(device)
            y = y.to(device)

            pred = model(X)
            loss = loss_(pred,y)
            test_error += loss.item()
            correct += (pred.argmax(1)==y).type(torch.float).sum().item()
    print(f'test_correct {correct} out of {size} and  Accuracy is{correct/size*100}' )
    test_accuracy = correct / size
    test_loss = test_error / len(data)
    writer.add_scalar("Test Accuracy", test_accuracy, epoch)
    writer.add_scalar("Test Loss", test_loss, epoch)
    print('test error :::',test_error)
    return test_error



> **train and test our model for 30 epochs******

In [ ]:
epoch = 30
for i in range(epoch):
    print('epoch number is ',i+1)
    train(traindata_loader,model,optimizer,loss_fn,i)
    test(model,testdata_loader,loss_fn,i)

end = time.time()
print((end-start)/60)

# SAVE MODEL

In [ ]:
!pwd

In [ ]:
torch.save(model.state_dict(), '/kaggle/working/transfer_learning.pth')


# LOADING MODEL

In [ ]:
model = models.resnet50(pretrained=True)
# limiting the weights to be constant during training
for param in model.parameters():
    param.requires_grad = False
model.fc = nn.Sequential(nn.Linear(2048, 256),nn.Dropout2d(.3),nn.ReLU(inplace=True),nn.Linear(256,64),nn.Dropout2d(.3),nn.Linear(64,2))


model.load_state_dict(torch.load('/kaggle/working/transfer_learning.pth'))

In [ ]:
model.to('cuda')

In [ ]:
def test(model,data,loss_,epoch):
    
    model.eval()
    test_error,correct = 0,0
    size  = len(data.dataset)

    with torch.no_grad():

        for batch,(X,y) in enumerate(data):
            
            X = X.to(device)
            y = y.to(device)

            pred = model(X)
            loss = loss_(pred,y)
            test_error += loss.item()
            correct += (pred.argmax(1)==y).type(torch.float).sum().item()
    print(f'test_correct {correct} out of {size} and  Accuracy is{correct/size*100}' )
    test_accuracy = correct / size
    test_loss = test_error / len(data)
    writer.add_scalar("Test Accuracy", test_accuracy, epoch)
    writer.add_scalar("Test Loss", test_loss, epoch)
    print('test error :::',test_error)
    return test_error



In [ ]:
test(model,testdata_loader,loss_fn,1)
